In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PowerTransformer
from kmodes.kprototypes import KPrototypes
import pickle
import joblib
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from matplotlib import pyplot

#### 모델 생성 후 저장

In [2]:
data = pd.read_csv('prototype_9.csv', encoding='cp949')

data=data.sample(frac=1).reset_index(drop=True)

#csv 파일을 copy하여 불량유무를 없앤 data를 x에 저장하고 target인 불량유무를 y에 저장
copy = data
y = data['불량유무']

copy.drop(['불량유무'], axis=1, inplace=True)

# attribute들을 X에 저장시킨 뒤 확인
X = copy

# extra tree model(80개의 tree 사용)

model = ExtraTreesClassifier(n_estimators=80, random_state=0)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

#모델 학습 후 저장
model.fit(X, y)
joblib.dump(model, 'Extra_tree_model.pkl')

['Extra_tree_model.pkl']

In [ ]:
# extra tree 내에서 사용할 tree 개수를 정하는 과정

# get a list of models to evaluate
def get_models():
    models = dict()
    # define number of trees to consider
    n_trees = [50,60,70,80,90,100]
    for n in n_trees:
        models[str(n)] = ExtraTreesClassifier(n_estimators=n)
    return models
 
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    # define the evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model and collect the results
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores
 

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    # evaluate the model
    scores = evaluate_model(model, X, y)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize the performance along the way
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

#### 군집화 -> 입력 -> 예측

In [2]:
# 입력을 받아 군집을 구하기 위한 파일 불러오기

old_file = pd.read_csv("본 파일.csv", encoding='cp949')

old_file.drop(['불량유무'], axis=1, inplace=True)

row = []

print("[재료명, 모델, 품명, 색상, Cavity, 사출속도, 사출압력, 사출위치, 사출시간, 보압속도, 보압압력, 보압시간, 계랸속도, 계량위치, 석백, 계량시간, 상부 금형 온도, 하부 금형 온도, 냉각 값, 쿠션 값, 실린더온도_노즐, 실린더온도(1, 2, 3, 4)를 입력해주세요]")

# 범주형 변수 입력
x = input('재료명을 입력해주세요 : ')
row.append(x)
x = input('모델을 입력해주세요 : ')
row.append(x)
x = input('품명을 입력해주세요 : ')
row.append(x)

# 수치형 변수 입력
x = int(input('색상이 투명이면 1, 그렇지 않으면 0을 입력해주세요 : '))
row.append(x)
x = int(input('cavity 수를 입력해주세요 : '))
row.append(x)
x = float(input('사출속도를 입력해주세요 : '))
row.append(x)
x = float(input('사출압력을 입력해주세요 : '))
row.append(x)
x = float(input('사출위치를 입력해주세요 : '))
row.append(x)
x = float(input('사출시간을 입력해주세요 : '))
row.append(x)
x = float(input('보압속도를 입력해주세요 : '))
row.append(x)
x = float(input('보압압력를 입력해주세요 : '))
row.append(x)
x = float(input('보압시간을 입력해주세요 : '))
row.append(x)
x = float(input('계량속도를 입력해주세요 : '))
row.append(x)
x = float(input('계량위치를 입력해주세요 : '))
row.append(x)
x = float(input('계량위치(석백)를 입력해주세요 : '))
row.append(x)
x = float(input('계량시간을 입력해주세요 : '))
row.append(x)
x = float(input('상부 금형 온도를 입력해주세요 : '))
row.append(x)
x = float(input('하부 금형 온도를 입력해주세요 : '))
row.append(x)
x = float(input('냉각 값을 입력해주세요 : '))
row.append(x)
x = float(input('쿠션 값을 입력해주세요 : '))
row.append(x)
x = float(input('실린더온도_노즐을 입력해주세요 : '))
row.append(x)
x = float(input('실린더온도_H1을 입력해주세요 : '))
row.append(x)
x = float(input('실린더온도_H2을 입력해주세요 : '))
row.append(x)
x = float(input('실린더온도_H3을 입력해주세요 : '))
row.append(x)
x = float(input('실린더온도_H4을 입력해주세요 : '))
row.append(x)

#마지막 행에 append한 후 tail로 확인
old_file = old_file.append(pd.Series(row, index=old_file.columns), ignore_index=True)

catColumnsPos = [old_file.columns.get_loc(col) for col in list(old_file.select_dtypes('object').columns)]

dfMatrix = old_file.to_numpy() 

#kprototype을 돌려 새로이 입력으로 받은 행의 군집 확인
kprototype = KPrototypes(n_jobs = -1, n_clusters = 9, init = 'Huang', random_state = 0) 
kprototype.fit_predict(dfMatrix, categorical = catColumnsPos)

# 9개의 군집으로 나뉘게 되고, 0~8까지의 label을 붙여줌
old_file['Labels'] = kprototype.labels_ 

# 군집으로 나누었으므로 categorical 변수를 전부 drop
old_file.drop(['재료명', '모델', '품명'], axis=1, inplace=True)

# 입력으로 받았던 row를 다시 불러옴
new_row = old_file.tail(n=1)

X = new_row.values.tolist()

# 모델 불러오기
model = joblib.load('Extra_tree_model.pkl')

[재료명, 모델, 품명, 색상, Cavity, 사출속도, 사출압력, 사출위치, 사출시간, 보압속도, 보압압력, 보압시간, 계랸속도, 계량위치, 석백, 계량시간, 상부 금형 온도, 하부 금형 온도, 냉각 값, 쿠션 값, 실린더온도_노즐, 실린더온도(1, 2, 3, 4)를 입력해주세요]
재료명을 입력해주세요 : 1
모델을 입력해주세요 : 2
품명을 입력해주세요 : 3
색상이 투명이면 1, 그렇지 않으면 0을 입력해주세요 : 4
cavity 수를 입력해주세요 : 1
사출속도를 입력해주세요 : 2
사출압력을 입력해주세요 : 3
사출위치를 입력해주세요 : 4
사출시간을 입력해주세요 : 1
보압속도를 입력해주세요 : 2
보압압력를 입력해주세요 : 3
보압시간을 입력해주세요 : 44
계량속도를 입력해주세요 : 1
계량위치를 입력해주세요 : 2
계량위치(석백)를 입력해주세요 : 3
계량시간을 입력해주세요 : 4
상부 금형 온도를 입력해주세요 : 1
하부 금형 온도를 입력해주세요 : 2
냉각 값을 입력해주세요 : 3
쿠션 값을 입력해주세요 : 4
실린더온도_노즐을 입력해주세요 : 1
실린더온도_H1을 입력해주세요 : 2
실린더온도_H2을 입력해주세요 : 3
실린더온도_H3을 입력해주세요 : 4
실린더온도_H4을 입력해주세요 : 1


In [3]:
# 모델 예측
print(np.where(model.predict(X) < 1, '입력하신 값으로 사출을 할 때엔 양품일 확률이 높습니다.','입력하신 값으로 사출을 할 때엔 불량일 확률이 높습니다.'))

['입력하신 값으로 사출을 할 때엔 불량일 확률이 높습니다.']
